In [1]:
import torch
import os
import sys
sys.path.insert(0, '../src')

from transformers import DonutProcessor
from modeling_tabeleiro import TabeleiroModel
from PIL import Image


IMG_PATH = "../../pubtabnet/imgs/final_eval/"

filelist = os.listdir(IMG_PATH)

2023-11-02 11:58:53.725996: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 11:58:53.726020: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 11:58:53.726037: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 11:58:53.730471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 11:58:54.313657: W tensorflow/compiler/

In [2]:
model = TabeleiroModel.from_pretrained("tabeleiro/")

model.eval()

You are using a model of type dimbart to instantiate a model of type mbart. This is not supported for all configurations of models and can yield errors.


TabeleiroModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
    

In [3]:
model.config

VisionEncoderDecoderConfig {
  "_commit_hash": null,
  "decoder": {
    "_name_or_path": "tabeleiro//dimbart_decoder",
    "activation_dropout": 0.0,
    "activation_function": "gelu",
    "add_cross_attention": true,
    "add_final_layer_norm": true,
    "architectures": [
      "MBartForCausalLM"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 1024,
    "decoder_attention_heads": 16,
    "decoder_ffn_dim": 4096,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 4,
    "decoder_start_token_id": null,
    "dim_max_position_embeddings": [
      2048,
      200,
      200
    ],
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stopping": false,
    "encoder_attention_heads": 16,
    "encoder_ffn_dim": 4096,
    "encoder_layerdrop": 0.0,
    "encoder_layers": 

In [4]:
processor = DonutProcessor.from_pretrained("../../pubtabnet/Donut_PubTables_TML_Processor")

In [5]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class DonutTableDataset(Dataset):
    def __init__(
        self,
        filelist
    ):
        self.filelist = filelist
        
        
    def __len__(self):
        return len(self.filelist)
    
    
    def __getitem__(self, idx):
        
        file_name = self.filelist[idx]
        
        image = Image.open(IMG_PATH + file_name)
        
        
        # inputs
        pixel_values = processor(image.convert("RGB"), random_padding=False, return_tensors="pt").pixel_values.squeeze()
        pixel_values = pixel_values.squeeze()
        
        encoding = dict(file_name = file_name, pixel_values=pixel_values)
        
        return encoding

In [6]:
test_set = DonutTableDataset(filelist)

In [7]:
cell_types = ["<cell>", "<col_header>", "<row_header>", "<row_and_col_header>"]
content_types = ["<content_row_and_col_header>", "<content_row_header>", "<content_col_header>", "<content>"]


added_content = ["1","<b>", "<i>", "<sup>", "<sub>","</b>", "</i>", "</sup>", "</sub>", "≥","≤"]

for i in range(2):
    for j in range(2):
        for k in range(2):
            cell_types.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            cell_types.append("<span_type=1" + str(i) + str(j) + str(k) + ">")


def token2cell(processor, seq, i, cell_coord, cell_type):
    if cell_type[:5] == "<span": 
        span_type = cell_type[1:-1]
        cell_type = processor.decode(seq[i])
        if cell_type[:5] == "<cont":
            i += 1
            if cell_type != "<content>":
                cell_type = "<"+cell_type[9:]
            else:
                cell_type = "cell"
    else:
        span_type = "span_type=0000"
    
    x, y = cell_coord
    aux_cell = {
        "row": x,
        "col": y,
        "col_header": cell_type in ["<col_header>", "<row_and_col_header>"],
        "row_header": cell_type in ["<row_header>", "<row_and_col_header>"],
        "colspan": -1 if span_type != "span_type=0000"  else 1,
        "rowspan": -1 if span_type != "span_type=0000"  else 1,
        "span_type": span_type,
        "content": ""
    }
    start_cont = i
    while processor.decode(seq[i]) not in ["<row>", "</s>", "<table>"] + cell_types:
        if(processor.decode(seq[i]) in added_content):
            aux_cell['content'] += processor.decode(seq[start_cont: i])+processor.decode(seq[i])
            start_cont = i+1
        i += 1
        
    aux_cell['content'] += processor.decode(seq[start_cont: i])
    
    return aux_cell, i

def token2row(processor, seq, i, row_id):
    cells = []
    while processor.decode(seq[i]) not in ["<row>", "</s>", "<table>"]:
        if processor.decode(seq[i]) in cell_types:
            aux_cell, i = token2cell(processor, seq, i+1, (row_id, len(cells)), processor.decode(seq[i]))
            cells.append(aux_cell)
        else:
            i += 1
    return cells, i


def token2table(processor, seq, i):
    rows = []
    while processor.decode(seq[i]) not in ["</s>", "<table>"]:
        if processor.decode(seq[i]) == "<row>":
            aux_row, i = token2row(processor, seq, i+1, len(rows))
            rows.append(aux_row)
        else:
            i += 1
            
    return rows, i


def crop_empty_left(table):
    for row in table:
        empty_left = []
        for cell in row:
            if(cell['content'] == "" and cell['colspan'] == 1):
                empty_left.append(cell)
            else:
                empty_left.clear()
        
        for cell in empty_left:
            cell['colspan'] = 0
            cell['rowspan'] = 0

def token2ann(processor, seq, i):
    tables = []
    while processor.decode(seq[i]) != "</s>":
        if processor.decode(seq[i] == "<table>"):
            aux_table, i = token2table(processor, seq, i+1)
            #crop_empty_left(aux_table)
            tables.append(aux_table)
        else:
            i += 1
        
    return {'tables': tables}

In [8]:
def update_vals(cell, content, col_header, row_header):
    cell['colspan'] = 0
    cell['rowspan'] = 0
    
    if cell['content'] != "":
        content = cell['content']
    
    col_header = col_header or cell['col_header']
    row_header = row_header or cell['row_header']
    
    return content, col_header, row_header


def define_by_path(cell, table):
    col_header, row_header = False, False
    i, j = cell['row'], cell['col']
    max_i, max_j = i, j
    first_j = j

    content = ''
    
    while True:
        while True:
            content, col_header, row_header = update_vals(cell, content, col_header, row_header)
            
            if(cell['span_type'][-4:-3] != '1'):
                break
            j += 1
            if j >= len(table[i]):
                break
            cell = table[i][j]
            
        if(cell['span_type'][-3:-2] != '1'):
            break
        
        i += 1
        j = first_j
        
        if i >= len(table) or j >= len(table[i]):
            break
        max_i, max_j  = max(i, max_i), max(j, max_j)
        cell = table[i][j]
    
    return (max_i, max_j), content, col_header, row_header

    
def define_spannings(table):
    for row in table:
        for cell in row:
            if(cell['colspan'] == -1):
                end_coord = (cell['row'], cell['col'])
                content = cell['content']
             
                end_coord, content, col_header, row_header = define_by_path(cell, table)
                
                
                cell['rowspan'] = end_coord[0] - cell['row'] + 1
                cell['colspan'] = end_coord[1] - cell['col'] + 1
                cell['col_header'] = col_header
                cell['row_header'] = row_header
                cell['content'] = content
                

In [9]:
def cell2html(cell):
    seq = "<td"
    if(cell['rowspan'] > 1):
        seq += ' rowspan="' + str(cell['rowspan']) +'"'
    if(cell['colspan'] > 1):
        seq += ' colspan="' + str(cell['colspan']) +'"'
    seq += ">" + cell["content"] + "</td>"
    
    return seq

def table2html(table):
    seq = ""
    head = False
    body = False
    for row in table:
        row_seq = ""
        count_header = 0
        row_len = len(row)
        for cell in row:
            if(cell['colspan'] > 0):
                count_header += cell['col_header']
                row_seq += cell2html(cell)
            else:
                row_len -= 1
            
        if(count_header  > row_len/2):
            if body:
                seq += "</tbody>"
                body = False
                
            if not head:
                seq += "<thead>"
                head = True
            seq +=  "<tr>" + row_seq + "</tr>" 
        else:
            if head:
                seq += "</thead>"
                head = False
                
            if not body:
                seq += "<tbody>"
                body = True
                
            seq +=  "<tr>" + row_seq + "</tr>"
    
    if head:
        seq += "</thead>"
        
    if body:
        seq += "</tbody>"
    
    return seq

In [10]:
processor.decode(processor.tokenizer.eos_token_id)

'</s>'

In [11]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(test_set, batch_size=1, shuffle=True)

In [ ]:
import torch
from tqdm.auto import tqdm

out_dics = {}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# prepare decoder inputs
task_prompt = "<table_extraction>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
decoder_input_ids = decoder_input_ids.to(device)

for i, batch in enumerate(tqdm(train_dataloader)):
    
    pixel_values = batch["pixel_values"].to(device)
    filename = batch["file_name"][0]
    
    # autoregressively generate sequence
    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length= 2048,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams= 1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
        )
    
    seq = torch.cat((outputs.sequences[0], torch.Tensor([2, 2]).int().to(device)), 0)
    
    table = token2ann(processor, seq, 3)
    define_spannings(table['tables'][0])
    
    if len(outputs.sequences[0]) > 2049:
        print(processor.tokenizer.decode(outputs.sequences[0]))
        print(len(outputs.sequences[0]))
        print(table2html(table['tables'][0]))
        print(filename)
        raise ah
    try:
        define_spannings(table['tables'][0])
    except Exception:
        print(filename)
        print(table2html(table['tables'][0]))
    
    out_dics[filename] = table2html(table['tables'][0])

  0%|          | 0/9064 [00:00<?, ?it/s]

In [ ]:
processor.tokenizer.decode(outputs.sequences[0])

In [ ]:
cp_dic = out_dics.copy()

In [ ]:
out_dics = cp_dic.copy()
for filename in out_dics:
    out_dics[filename] = "<html><body><table>" + out_dics[filename] + "</table></body></html>"

In [ ]:
import json
with open("../../pubtabnet/TML_pred_dic-EPOCH1.json", 'w') as out:
    json.dump(out_dics, out, ensure_ascii=False)

In [ ]:
print(torch.cat((outputs.sequences[0], torch.tensor([processor.tokenizer.eos_token_id]).to(device)), 0))

In [ ]:
import json
with open("../../pubtabnet/final_eval.json", encoding="utf-8") as f:
    gts = json.load(f)

for gt in gts:
    if(gt == "663f4502ef940b47563185fb6dd16307b43b895fdb4fe1bbe8e514e6ad2bf6f2.png"):
        print(gts[gt]['html'].replace("<b>", "").replace("</b>", ""))

In [ ]:
del model

In [ ]:
processor